## Alma API Students

Writing a code that communicates with the Alma url endpoint via an api to return student demographic data at each campus and uplaoding data extracted from the API to a BigQuery data warehousing software.

In [1]:
#importing relevant packages
from google.cloud import bigquery #--> python client for bigquery
import pandas as pd
import requests
import json
from requests.auth import HTTPDigestAuth
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv
import os

In [2]:
env_file_path = '/Users/scipio/Alma_API_Scripts/ALMA_API.env'

In [3]:
load_dotenv(dotenv_path = env_file_path )

True

In [4]:
# retrieving releavnt variables from the .env file
API_KEY = os.getenv('API_KEY')
AUTH_SECRET = os.getenv('AUTH_SECRET')

**FCA Current School Year**: 65e8a8461e0c3dd517076bcf 

**363 Current School Year ID**: 664cc55127c6b4a81806658b

In [5]:
# setting up BigQuery authentication
credentials = service_account.Credentials.from_service_account_file(
    '/Users/scipio/Downloads/single-being-353600-82aaccaecf53.json'
                                                                   )
#initializing BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

### Extraction of Student Data FCA

In [6]:
# Defining credential api_key and auth_secret 
api_key = API_KEY
auth_secret = AUTH_SECRET


# URL of the API endpoint FCA 2024-2025 SY
url= 'https://facs.api.getalma.com/v2/fca/students?schoolYearId=65e8a8461e0c3dd517076bcf' 

# Headers
headers = {
    'Content-Type':'application/json',
    'Accept':'application/json, application/problem+json'
}

# Make the GET request with Digest Authentication
response_fca = requests.get(url, headers=headers, auth=HTTPDigestAuth(api_key, auth_secret))

# Making response into a json object
r_fca = response_fca.json()

# Extracting value from r_fca
for key,value in r_fca.items():
    value #--> list of dictionaries

# coverting python object to a dataframe
df_r_fca = pd.DataFrame(value)

#data cleaning
df_r_fca = df_r_fca[df_r_fca['graduationYear']>2024] # --> filtering out seniors
df_r_fca['race'] = df_r_fca['race'].str[0] #--> making race a string
df_r_fca['languages'] = df_r_fca['languages'].str[0] #--> making langauges a string
df_r_fca['id'] = df_r_fca['id'].str.strip()


# function for race column
def race(row):
    
    """
    If a person's ehtincity is 'Hispanic Or Latino' so is their race
    """
    if row['ethnicity'] == 'Hispanic Or Latino':
        return 'Hispanic Or Latino'
    else:
        return row['race']
    
df_r_fca['race'] = df_r_fca.apply(race, axis = 1)

### Extraction of Student Data 363

In [7]:
# Defining credential api_key and auth_secret 
api_key = API_KEY
auth_secret = AUTH_SECRET

# URL of the API endpoint FACS 2024-2025 SY
url= 'https://facs.api.getalma.com/v2/facs363/students?schoolYearId=664cc55127c6b4a81806658b' 

# Headers
headers = {
    'Content-Type':'application/json',
    'Accept':'application/json, application/problem+json'
}

# Make the GET request with Digest Authentication
response_363 = requests.get(url, headers=headers, auth=HTTPDigestAuth(api_key, auth_secret))

# Making response into a json object
r_363 = response_363.json()

# extracting value from r_363
for key,value in r_363.items():
    value #--> list of dictionaries
    

# coverting python object to a dataframe
df_r_363 = pd.DataFrame(value)

#data cleaning
df_r_363['race'] = df_r_363['race'].str[0]
df_r_363['languages'] = df_r_363['languages'].str[0]
df_r_363['id'] = df_r_363['id'].str.strip()

# function for race column
def race(row):
    
    """
    If a person's ehtincity is 'Hispanic Or Latino' so is their race
    """
    if row['ethnicity'] == 'Hispanic Or Latino':
        return 'Hispanic Or Latino'
    else:
        return row['race']
    
df_r_363['race'] = df_r_363.apply(race, axis = 1)

### Transformation

In [8]:
#concating df_r_fca and df_r_363
df = pd.concat([df_r_fca,df_r_363])

# returning active or not active students
df = df[(df['status']=='Active')|(df['status']=='Not Activated')]

# retunring relevant columns
df = df[['id','schoolId','stateId','firstName', 'lastName','gender', 
    'ethnicity', 'race', 'languages','dob','graduationYear', 'status']]

# filteirng out duplicated values 
df = df[~df.duplicated()]

In [9]:
df.shape

(1141, 12)

In [10]:
df.head()

,id,schoolId,stateId,firstName,lastName,gender,ethnicity,race,languages,dob,graduationYear,status
0,60ca0e8a004d271aa50c7693,2612016,6684846964,Daniel,Young,Male,Not Specified,Black or African American,English,2008-05-07,2026.0,Active
1,60ca0e8abbf8f9301c6331b2,1422015,6053098154,Aurelio,Williams,Male,Hispanic Or Latino,Hispanic Or Latino,English,2006-10-23,2026.0,Active
2,60ca0e8a6ce99937aa1276a0,2102015,8038168482,Kaliyah,Shy,Female,Not Specified,Black or African American,English,2010-07-21,2028.0,Active
3,60ca0e8a30c7b82ad6073582,3332015,3006053492,Gerall,Aguero,Male,Hispanic Or Latino,Hispanic Or Latino,English,2008-10-27,2027.0,Active
5,60ca0e8be34ae24d802e9c17,2302018,2661826754,Briana,Thomas,Female,Not Specified,Black or African American,English,2008-08-06,2026.0,Active


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1141 entries, 0 to 1277
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1141 non-null   object 
 1   schoolId        1141 non-null   object 
 2   stateId         970 non-null    object 
 3   firstName       1141 non-null   object 
 4   lastName        1141 non-null   object 
 5   gender          1141 non-null   object 
 6   ethnicity       1141 non-null   object 
 7   race            1139 non-null   object 
 8   languages       960 non-null    object 
 9   dob             1141 non-null   object 
 10  graduationYear  952 non-null    float64
 11  status          1141 non-null   object 
dtypes: float64(1), object(11)
memory usage: 115.9+ KB


### Load

In [12]:
# loading into BigQuery database
table_id = 'Alma_Data_API.Students'#--> dataset id and table name

df.to_gbq(table_id, project_id=credentials.project_id, if_exists='replace', credentials=credentials)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4084.04it/s]
